# Accurate prediction of developer salaries from 2017 stackoverflow developer survey data

### Main Questions: Would the prediction result be different if I use get dummies func differently?

#### Q1. If you use the get dummie function according to the lecture method, the number of columns to increase is too large. Is there any way to reduce this?

   
#### Q2. What is the difference between the existing method of finding a dummy and the result when using my method?  


#### Q3. What can be done to improve the results?


#### Q4. Comparison conclusion

# 1. Business Understanding
### I want to investigate the annual income of developers based on stackoverflow survey 2017 data for a project of data science course taken at udacity.

# 2. Data Understanding

## 2.1. Import libraries

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2.2 Import data

In [10]:
def import_data(data_path, drop_col=[]):
    df = pd.read_csv(data_path)
    print("Imported dataframe shape: ", df.shape)
    
    # drop specified columns
    df = df.drop(drop_col, axis=1)
    
    # drop column with all nan values
    df = df.dropna(how='all', axis=1)
    print(f"Df now has shape of {df.shape}")
    print(f"Importing df with shape of {df.shape} is complete!!!")
    return df

In [11]:
df = pd.read_csv('./survey_results_public_2017.csv')
print(f"df shape: {df.shape}")
df.head()

df shape: (51392, 154)


,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,...,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Strongly disagree,Male,High school,White or of European descent,Strongly disagree,Strongly agree,Disagree,Strongly agree,NaN,NaN
1,2,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,...,Strongly disagree,Male,A master's degree,White or of European descent,Somewhat agree,Somewhat agree,Disagree,Strongly agree,NaN,37500.0
2,3,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A professional degree,White or of European descent,Somewhat agree,Agree,Disagree,Agree,113750.0,NaN
3,4,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A doctoral degree,White or of European descent,Agree,Agree,Somewhat agree,Strongly agree,NaN,NaN
4,5,Professional developer,"Yes, I program as a hobby",Switzerland,No,Employed full-time,Master's degree,Computer science or software engineering,Never,10 to 19 employees,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.3 Calculate ratio of nan values in Salary column

In [12]:
salary_nan_ratio = df.Salary.isnull().sum()/df.shape[0]
print("nan ratio of Salary column is ", salary_nan_ratio)
print(f"number of rows after drop: {df.dropna(axis=0, subset='Salary').shape[0]}")

nan ratio of Salary column is  0.7491632938978829
number of rows after drop: 12891


## 2.4 Multiple values in cells and get dummies
#### Cells of some columns, which are categorical datatypes, have multiple values and are separated by semi colons(;). In this case, when performing one hot encoding using pd.get_dummies function, a value of 1 should be given to all the corresponding columns, not just one column.

### Example of column with multiple values

In [13]:
print(df.HaveWorkedLanguage.shape)
df.HaveWorkedLanguage[:4]

(51392,)


0                                                Swift
1                        JavaScript; Python; Ruby; SQL
2                                    Java; PHP; Python
3                               Matlab; Python; R; SQL
4                                                  NaN
5                                JavaScript; PHP; Rust
6                                       Matlab; Python
7    CoffeeScript; Clojure; Elixir; Erlang; Haskell...
8                                       C#; JavaScript
9                                   Objective-C; Swift
Name: HaveWorkedLanguage, dtype: object

### dummy will look like below, and it doesn't look good

In [14]:
dummy = pd.get_dummies(df.HaveWorkedLanguage)
print(dummy.shape)
dummy.head(3)

(51392, 8438)


,Assembly,Assembly; C,Assembly; C#,Assembly; C#; Clojure,Assembly; C#; CoffeeScript; Dart; Go; Haskell; Java; JavaScript; Lua; Matlab; PHP; Python; R; Ruby; Rust; Smalltalk; SQL; TypeScript,Assembly; C#; CoffeeScript; JavaScript; SQL; Swift; TypeScript,Assembly; C#; Common Lisp; Elixir; F#,Assembly; C#; Common Lisp; Elixir; Groovy; R; Rust; Smalltalk; VB.NET; Visual Basic 6,Assembly; C#; Common Lisp; JavaScript; PHP; Python; SQL; VBA,Assembly; C#; F#; Java; JavaScript; Objective-C; PHP; Python; SQL; Swift; VB.NET; VBA,...,Swift; TypeScript,Swift; VB.NET,TypeScript,VB.NET,VB.NET; VBA,VB.NET; VBA; Visual Basic 6,VB.NET; Visual Basic 6,VBA,VBA; Visual Basic 6,Visual Basic 6
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### It seems that number of columns in dummy is too large. I think If the number of columns can be reduced, better results can be obtained.

# 3. Data Preparation
#### 1. If the value of the salary column is nan, drop the row

#### 2. Remove unnecessary columns or columns with all values of nan

#### 3. If the data type of the column is numeric, fill in the nan value with the average value.

#### 4. If the data type of the column is object, do get dummy

## 3.1. If the value of the salary column is nan, drop the row. Use Salary column as y(label)

In [15]:
print(df.shape)
df = df.dropna(subset='Salary', axis=0)
df.shape

(51392, 154)


(12891, 154)

In [16]:
y = df.Salary
X = df.drop('Salary', axis=1)
print(y.shape)
print(X.shape)

(12891,)
(12891, 153)


## 3.2. Remove unnecessary columns or columns with all values of nan

In [17]:
# Drop unnecessary
print(X.shape)
X = X.drop('Respondent', axis=1)
print(df.shape)
X = X.dropna(how='all', axis=1)
print(X.shape)

(12891, 153)
(12891, 154)
(12891, 145)


## 3.3. If the data type of the column is numeric, fill in the nan value with the average value.

In [18]:
def get_numeric_type_columns(df):
    df_num = df.select_dtypes(include=['int', 'float'])
    df_num_cols = df_num.columns
    print("Numeric columns from this df: ", df_num.shape[1])
    return df_num_cols

In [19]:
def fill_mean(series):
    # Mean function
    mean = series.mean()
    # Fill the mean
    series = series.fillna(mean, axis=0)
    return series

In [20]:
def get_numeric_dict(df):
    df_num_cols = get_numeric_type_columns(df)
    numeric_df_dict = {}
    numeric_df_names_list = []
    for col in df_num_cols:
        df_filled = fill_mean(df[col])
        numeric_df_dict[col] = df_filled
        numeric_df_names_list.append(col)
    return numeric_df_dict, numeric_df_names_list

### I will save each columns result in dictionary for later use

In [21]:
numeric_df_dict, numeric_df_names_list = get_numeric_dict(X)
numeric_df_names_list

Numeric columns from this df:  4


['CareerSatisfaction',
 'JobSatisfaction',
 'HoursPerWeek',
 'StackOverflowSatisfaction']

## 3.4. If the data type of the column is object, get dummies

In [23]:
def get_object_type_columns(df):
    df_obj = df.select_dtypes(include=['object'])
    df_obj_cols = df_obj.columns
    print("Object columns from this df: ", df_obj.shape[1])
    return df_obj_cols

In [24]:
def get_multiple_value_columns(df):
    contain_semicolons = df.apply(lambda col: 
                                  col.str.contains(';').any()
                                  if col.dtypes==object
                                  else False)
    df_contain_semicolons = df.loc[:, contain_semicolons]
    multiple_value_columns = df_contain_semicolons.columns
    return multiple_value_columns

In [25]:
def get_dummy_simple(series, dummy_na=False):
    df = pd.DataFrame()
    try:
        df = pd.get_dummies(series, prefix=series.name, prefix_sep='_', drop_first=False, dummy_na=dummy_na)
    except:
        print(f"Error with get_dummy_simple(), column: {series.name}")
    return df

In [26]:
def categorize_feature(series):
    series = series.apply(lambda value: list(value.split(';'))
                       if type(value)!=float
                       else float('nan'))
    return pd.get_dummies(series.apply(pd.Series).stack(dropna=False), prefix=series.name, prefix_sep='_').sum(level=0)    

In [27]:
def get_dummy_dict(df, multiple_value_columns=[]):
    df_obj_cols = get_object_type_columns(df)
    categorized_df_dict = {}
    categorized_df_names_list = []
    for col in df_obj_cols:
        if col in multiple_value_columns:
            df_dummy = categorize_feature(df[col]) # my method
        else:
            df_dummy = get_dummy_simple(df[col]) # simple method
        categorized_df_dict[col] = df_dummy
        categorized_df_names_list.append(col)
    return categorized_df_dict, categorized_df_names_list

In [28]:
multiple_value_columns = get_multiple_value_columns(X)
multiple_value_columns

Index(['DeveloperType', 'MobileDeveloperType', 'ImportantBenefits',
       'JobProfile', 'EducationTypes', 'SelfTaughtTypes', 'CousinEducation',
       'HaveWorkedLanguage', 'WantWorkLanguage', 'HaveWorkedFramework',
       'WantWorkFramework', 'HaveWorkedDatabase', 'WantWorkDatabase',
       'HaveWorkedPlatform', 'WantWorkPlatform', 'IDE', 'Methodology',
       'MetricAssess', 'StackOverflowDevices', 'Gender', 'Race'],
      dtype='object')

In [29]:
dummy_simple_df_dict, dummy_simple_df_names_list = get_dummy_dict(X)

Object columns from this df:  141


In [30]:
dummy_my_df_dict, dummy_my_df_names_list = get_dummy_dict(X, multiple_value_columns)

Object columns from this df:  141


In [32]:
assert X.shape[1] == len(numeric_df_names_list) + len(dummy_simple_df_names_list)
assert X.shape[1] == len(numeric_df_names_list) + len(dummy_my_df_names_list)

# 4. Modeling
### In this section I will find solutions to my questions

### Main Questions: Would the prediction result be different if I use get dummies func differently?

#### Q1. If you use the get dummie function according to the lecture method, the number of columns to increase is too large. Is there any way to reduce this?
   
#### Q2. What is the difference between the existing method of finding a dummy and the result when using my method?

#### Q3. What can be done to improve the results?

#### Q4. Comparison conclusion

## Q1. If you use the get dummie function according to the lecture method, the number of columns to increase is too large. Is there any way to reduce this?

### As shown in section 3(Data Preparation), I have come up with new method to get dummies

In [35]:
col = 'HaveWorkedLanguage'

In [36]:
print(dummy_simple_df_dict[col].shape)
dummy_simple = dummy_simple_df_dict[col]
dummy_simple.head(3)

(12891, 3445)


,HaveWorkedLanguage_Assembly,HaveWorkedLanguage_Assembly; C,HaveWorkedLanguage_Assembly; C#,HaveWorkedLanguage_Assembly; C#; CoffeeScript; JavaScript; SQL; Swift; TypeScript,HaveWorkedLanguage_Assembly; C#; Common Lisp; JavaScript; PHP; Python; SQL; VBA,HaveWorkedLanguage_Assembly; C#; Go; Hack; Haskell; JavaScript; Lua; Objective-C; Perl; PHP; Python; Ruby; Rust; SQL; Swift; TypeScript; VBA,HaveWorkedLanguage_Assembly; C#; Go; Java; JavaScript; PHP; SQL,HaveWorkedLanguage_Assembly; C#; Go; JavaScript; PHP; Ruby; SQL,HaveWorkedLanguage_Assembly; C#; Groovy; Java; JavaScript; Matlab; Objective-C; Python; R; Swift,HaveWorkedLanguage_Assembly; C#; Java; JavaScript; Lua; Objective-C; PHP; SQL,...,HaveWorkedLanguage_Scala; SQL; TypeScript,HaveWorkedLanguage_Scala; SQL; VB.NET; VBA,HaveWorkedLanguage_Smalltalk,HaveWorkedLanguage_Smalltalk; SQL,HaveWorkedLanguage_Smalltalk; TypeScript,HaveWorkedLanguage_Swift,HaveWorkedLanguage_Swift; TypeScript,HaveWorkedLanguage_VB.NET,HaveWorkedLanguage_VBA,HaveWorkedLanguage_Visual Basic 6
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
print(dummy_my_df_dict[col].shape)
dummy_my = dummy_my_df_dict[col]
dummy_my.head(3)

(12891, 68)


,HaveWorkedLanguage_ C,HaveWorkedLanguage_ C#,HaveWorkedLanguage_ C++,HaveWorkedLanguage_ Clojure,HaveWorkedLanguage_ CoffeeScript,HaveWorkedLanguage_ Common Lisp,HaveWorkedLanguage_ Dart,HaveWorkedLanguage_ Elixir,HaveWorkedLanguage_ Erlang,HaveWorkedLanguage_ F#,...,HaveWorkedLanguage_R,HaveWorkedLanguage_Ruby,HaveWorkedLanguage_Rust,HaveWorkedLanguage_SQL,HaveWorkedLanguage_Scala,HaveWorkedLanguage_Smalltalk,HaveWorkedLanguage_Swift,HaveWorkedLanguage_VB.NET,HaveWorkedLanguage_VBA,HaveWorkedLanguage_Visual Basic 6
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### My function has fewer columns and multiple columns can be selected

In [38]:
number_of_ones = dummy_simple.loc[14, :].sum()
print(f"Number of selected columns in dummy_simple is {number_of_ones}")

Number of selected columns in dummy_simple is 1


In [39]:
number_of_ones = dummy_my.loc[14, :].sum()
print(f"Number of selected columns in dummy_my is {number_of_ones}")

Number of selected columns in dummy_my is 3


# Q2. What is the difference between the existing method of finding a dummy and the result when using my method?

In [91]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import time

result_summary = [] # [[condition, rank_test, rank_train, time consumption], ...]

In [42]:
def train_model(X, y):
    lm_model = LinearRegression(normalize=True) # Instantiate
    lm_model.fit(X, y) #Fit
    return lm_model

In [145]:
def check_result(col, df, y):
#     print(col)
#     print(f"shape: {df.shape}")
    
    X = df      
    #Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42) 
    
    model = train_model(X_train, y_train)

    #Predict and score the model
    y_test_preds = model.predict(X_test) 
    y_train_preds = model.predict(X_train)
    test_score = r2_score(y_test, y_test_preds)
    train_score = r2_score(y_train, y_train_preds)
#     print(f"test_score: {test_score}, train_score: {train_score}")
#     print('-'*40)
    
    return test_score, train_score

## 2.1 Get prediction from each column

In [76]:
def get_rank_each_column(dict_num, dict_dummy):
    rank_test = []
    rank_train = []

    for col, df_num in dict_num.items():
        test_score, train_score = check_result(col, df_num.values.reshape(-1,1), y)

        rank_test.append([test_score, col])
        rank_train.append([train_score, col])

    for col, df_obj in dict_dummy.items():   
        test_score, train_score = check_result(col, df_obj, y)

        rank_test.append([test_score, col])
        rank_train.append([train_score, col])
    return rank_test, rank_train

### Simple dummy results for each columns

In [93]:
start_time = time.time()
rank_test, rank_train = get_rank_each_column(numeric_df_dict, dummy_simple_df_dict)
time_consumption = time.time()-start_time

CareerSatisfaction
shape: (12891, 1)
test_score: 0.02440744353670876, train_score: 0.02511874806277581
----------------------------------------
JobSatisfaction
shape: (12891, 1)
test_score: 0.015714880014184307, train_score: 0.013990287390995837
----------------------------------------
HoursPerWeek
shape: (12891, 1)
test_score: 0.008116429443099316, train_score: 0.005353632681996512
----------------------------------------
StackOverflowSatisfaction
shape: (12891, 1)
test_score: 0.005130504975913497, train_score: 0.005559666137063934
----------------------------------------
Professional
shape: (12891, 1)
test_score: -1.858952728239771e-05, train_score: 0.0
----------------------------------------
ProgramHobby
shape: (12891, 4)
test_score: 0.0005247655454900801, train_score: 0.0057765601272673495
----------------------------------------
Country
shape: (12891, 136)
test_score: -7.709702341110363e+25, train_score: 0.570935821010838
----------------------------------------
University
shape:

test_score: -3.038669656748752e+29, train_score: -0.45432784884518873
----------------------------------------
ClickyKeys
shape: (12891, 2)
test_score: 0.0009173493809646605, train_score: 0.0009201719340772296
----------------------------------------
JobProfile
shape: (12891, 280)
test_score: -1.756481845222238e+28, train_score: 0.07184415657926957
----------------------------------------
ResumePrompted
shape: (12891, 8)
test_score: 0.007348218600445988, train_score: 0.009252590135821137
----------------------------------------
LearnedHiring
shape: (12891, 8)
test_score: 0.010394993173299616, train_score: 0.013904729404609073
----------------------------------------
ImportantHiringAlgorithms
shape: (12891, 5)
test_score: 0.006151073898474579, train_score: 0.00920158875427457
----------------------------------------
ImportantHiringTechExp
shape: (12891, 5)
test_score: 0.016662468584125234, train_score: 0.01812199910832213
----------------------------------------
ImportantHiringCommunica

test_score: 0.01396088313618915, train_score: 0.012978908134480882
----------------------------------------
StackOverflowBetter
shape: (12891, 5)
test_score: 0.009390536736195942, train_score: 0.010785354820559978
----------------------------------------
StackOverflowWhatDo
shape: (12891, 5)
test_score: 0.006180239226589834, train_score: 0.01149144330948737
----------------------------------------
StackOverflowMakeMoney
shape: (12891, 5)
test_score: 0.00617275047883703, train_score: 0.010522676755073346
----------------------------------------
Gender
shape: (12891, 19)
test_score: 1.3587170536011328e-05, train_score: 0.008618156069942318
----------------------------------------
HighestEducationParents
shape: (12891, 10)
test_score: 0.011432648891307795, train_score: 0.030574280762974193
----------------------------------------
Race
shape: (12891, 55)
test_score: 0.06349849905100025, train_score: 0.06561597617228043
----------------------------------------
SurveyLong
shape: (12891, 5)
t

In [94]:
rank_test.sort(reverse=True)
print("r2 score", "            ", "Column name")
rank_test[:3]

r2 score              Column name


[[0.4167738538485196, 'Currency'],
 [0.20301667135459334, 'YearsCodedJob'],
 [0.17681542205634249, 'YearsProgram'],
 [0.10317553831646353, 'CompanyType'],
 [0.06349849905100025, 'Race'],
 [0.059509865901402725, 'University'],
 [0.05663789325191737, 'Overpaid'],
 [0.05192326010693504, 'CompanySize'],
 [0.038759220445147125, 'ImportantHiringPMExp'],
 [0.03699140020104186, 'StackOverflowCopiedCode']]

In [95]:
print(f"Best single column to predict the salary is {rank_test[0][1]}")
"Time consumption: ", time_consumption

Best single column to predict the salary is Currency


('Time consumption: ', 53.780603885650635)

In [98]:
# [condition, rank_test, rank_train, time consumption]
result_summary.append([["simple dummy", "single column"], rank_test, rank_train, time_consumption]) 

### my dummy results

In [99]:
start_time = time.time()
rank_test, rank_train = get_rank_each_column(numeric_df_dict, dummy_my_df_dict)
time_consumption = time.time() - start_time

CareerSatisfaction
shape: (12891, 1)
test_score: 0.02440744353670876, train_score: 0.02511874806277581
----------------------------------------
JobSatisfaction
shape: (12891, 1)
test_score: 0.015714880014184307, train_score: 0.013990287390995837
----------------------------------------
HoursPerWeek
shape: (12891, 1)
test_score: 0.008116429443099316, train_score: 0.005353632681996512
----------------------------------------
StackOverflowSatisfaction
shape: (12891, 1)
test_score: 0.005130504975913497, train_score: 0.005559666137063934
----------------------------------------
Professional
shape: (12891, 1)
test_score: -1.858952728239771e-05, train_score: 0.0
----------------------------------------
ProgramHobby
shape: (12891, 4)
test_score: 0.0005247655454900801, train_score: 0.0057765601272673495
----------------------------------------
Country
shape: (12891, 136)
test_score: -7.709702341110363e+25, train_score: 0.570935821010838
----------------------------------------
University
shape:

test_score: 0.0009173493809646605, train_score: 0.0009201719340772296
----------------------------------------
JobProfile
shape: (12891, 37)
test_score: 0.07310429022915554, train_score: 0.06828447343954602
----------------------------------------
ResumePrompted
shape: (12891, 8)
test_score: 0.007348218600445988, train_score: 0.009252590135821137
----------------------------------------
LearnedHiring
shape: (12891, 8)
test_score: 0.010394993173299616, train_score: 0.013904729404609073
----------------------------------------
ImportantHiringAlgorithms
shape: (12891, 5)
test_score: 0.006151073898474579, train_score: 0.00920158875427457
----------------------------------------
ImportantHiringTechExp
shape: (12891, 5)
test_score: 0.016662468584125234, train_score: 0.01812199910832213
----------------------------------------
ImportantHiringCommunication
shape: (12891, 5)
test_score: 0.026928444070211, train_score: 0.02748299420203615
----------------------------------------
ImportantHiringO

test_score: 0.03297184650898477, train_score: 0.03257472814823148
----------------------------------------
QuestionsInteresting
shape: (12891, 5)
test_score: 0.011268393328435988, train_score: 0.013669800908288177
----------------------------------------
QuestionsConfusing
shape: (12891, 5)
test_score: 0.0222046313266695, train_score: 0.025080474073352566
----------------------------------------
InterestedAnswers
shape: (12891, 5)
test_score: 0.00579591348183095, train_score: 0.009654643178323874
----------------------------------------


In [100]:
rank_test.sort(reverse=True)
print("r2 score", "            ", "Column name")
rank_test[:3]

r2 score              Column name


[[0.4167738538485196, 'Currency'],
 [0.20301667135459334, 'YearsCodedJob'],
 [0.17681542205634249, 'YearsProgram'],
 [0.10317553831646353, 'CompanyType'],
 [0.07708971568958778, 'IDE'],
 [0.07310429022915554, 'JobProfile'],
 [0.0709689802747091, 'ImportantBenefits'],
 [0.06621110184471446, 'Race'],
 [0.06473857270436212, 'HaveWorkedLanguage'],
 [0.059509865901402725, 'University']]

In [101]:
print(f"Best single column to predict the salary is {rank_test[0][1]}")
"Time consumption: ", time_consumption

Best single column to predict the salary is Currency


('Time consumption: ', 0.9999985694885254)

In [102]:
# [condition, rank_test, rank_train, time consumption]
result_summary.append([["my dummy", "single column"], rank_test, rank_train, time_consumption]) 

## 2.2 Get prediction from number of columns
#### Use more than 1 columns to predict the salary
#### Column selection criteria are sequentially selected by the number determined in the ranking.

In [106]:
num_features = [1, 2, 3, 4, 5, 10, 20, 50, 100, X.shape[1]]

In [107]:
def get_rank_number_column(dict_num, dict_dummy, rank):
    rank_test, rank_train = [], []
    for num in num_features:
        features = []
        for i in range(num):
            features.append(rank[i][1])
        X = pd.DataFrame()
        for feature in features:
            if feature in dict_num:
                X = pd.concat([X, dict_num[feature]], axis=1)
            elif feature in dict_dummy:
                X = pd.concat([X, dict_dummy[feature]], axis=1)
            else:
                print(f"{feature} not in numeric_df_dict or categorized_df_dict!!!")

        test_score, train_score = check_result(num, X, y)
        rank_test.append([test_score, num])
        rank_train.append([train_score, num])
    return rank_test, rank_train

### Simple dummy results for number of columns

In [108]:
start_time = time.time()
rank_test, rank_train = get_rank_number_column(numeric_df_dict, dummy_simple_df_dict, rank_test_simple)
time_consumption = time.time()-start_time

1
shape: (12891, 17)
test_score: 0.4167738538485196, train_score: 0.4339837071598497
----------------------------------------
2
shape: (12891, 38)
test_score: 0.5533608651193768, train_score: 0.5717896815799879
----------------------------------------
3
shape: (12891, 59)
test_score: 0.5579702570388851, train_score: 0.5791724605692948
----------------------------------------
4
shape: (12891, 70)
test_score: 0.5907981786641967, train_score: 0.6093640430709086
----------------------------------------
5
shape: (12891, 125)
test_score: -2.8933110678915737e+24, train_score: 0.6156692302749206
----------------------------------------
10
shape: (12891, 154)
test_score: -1.3860895637801805e+25, train_score: 0.655538856055528
----------------------------------------
20
shape: (12891, 211)
test_score: -6.550066498594873e+23, train_score: 0.6745769563524515
----------------------------------------
50
shape: (12891, 409)
test_score: -2.447397800897439e+26, train_score: 0.6949489154160645
---------

In [109]:
rank_test.sort(reverse=True)
print("r2 score", "            ", "Number of Columns")
rank_test[:3]

r2 score              Number of Columns


[[0.5907981786641967, 4],
 [0.5579702570388851, 3],
 [0.5533608651193768, 2],
 [0.4167738538485196, 1],
 [-0.42987120036592863, 145],
 [-6.550066498594873e+23, 20],
 [-2.8933110678915737e+24, 5],
 [-4.3798824048001777e+24, 100],
 [-1.3860895637801805e+25, 10],
 [-2.447397800897439e+26, 50]]

In [110]:
print(f"Best number of columns to predict the salary is {rank_test[0][1]}")
"Time consumption: ", time_consumption

Best number of columns to predict the salary is 4


('Time consumption: ', 241.86028480529785)

In [111]:
# [condition, rank_test, rank_train, time consumption]
result_summary.append([["my dummy", "number of columns"], rank_test, rank_train, time_consumption]) 

### my dummy results for number of columns

In [112]:
start_time = time.time()
rank_test, rank_train_number2 = get_rank_number_column(numeric_df_dict, dummy_my_df_dict, rank_test_my)
time_consumption = time.time()-start_time

1
shape: (12891, 17)
test_score: 0.4167738538485196, train_score: 0.4339837071598497
----------------------------------------
2
shape: (12891, 38)
test_score: 0.5533608651193768, train_score: 0.5717896815799879
----------------------------------------
3
shape: (12891, 59)
test_score: 0.5579702570388851, train_score: 0.5791724605692948
----------------------------------------
4
shape: (12891, 70)
test_score: 0.5907981786641967, train_score: 0.6093640430709086
----------------------------------------
5
shape: (12891, 113)
test_score: 0.6057809978127665, train_score: 0.6242068087506347
----------------------------------------
10
shape: (12891, 272)
test_score: -1.4121282631674564e+26, train_score: 0.6569482336363826
----------------------------------------
20
shape: (12891, 459)
test_score: -3.7539335231166577e+25, train_score: 0.7006932003351858
----------------------------------------
50
shape: (12891, 748)
test_score: -3.979132219365716e+24, train_score: 0.7265177408317718
------------

In [113]:
rank_test.sort(reverse=True)
print("r2 score", "            ", "Number of Columns")
rank_test[:3]

r2 score              Number of Columns


[[0.6057809978127665, 5],
 [0.5907981786641967, 4],
 [0.5579702570388851, 3],
 [0.5533608651193768, 2],
 [0.4167738538485196, 1],
 [-9.608937855305889e+23, 100],
 [-1.9138531130635174e+24, 145],
 [-3.979132219365716e+24, 50],
 [-3.7539335231166577e+25, 20],
 [-1.4121282631674564e+26, 10]]

In [114]:
print(f"Best number of columns to predict the salary is {rank_test[0][1]}")
"Time consumption: ", time_consumption

Best number of columns to predict the salary is 5


('Time consumption: ', 3.8835387229919434)

In [115]:
# [condition, rank_test, rank_train, time consumption]
result_summary.append([["my dummy", "number of columns"], rank_test, rank_train, time_consumption])

## Summary

In [128]:
condition, rank_test, _, time_consumption = result_summary[0]
print(f"Condition(get dummy method): use original get_dummies function")
print(f"Condition(number of columns): only 1 column is used to predict Salary")
print(f"The column to predict the best r2score: {rank_test[0][1]}, r2score: {round(rank_test[0][0], 3)}")
print(f"Time consumption: {round(time_consumption, 1)} seconds")

Condition(get dummy method): use original get_dummies function
Condition(number of columns): only 1 column is used to predict Salary
The column to predict the best r2score: Currency, r2score: 0.417
Time consumption: 53.8 seconds


In [129]:
condition, rank_test, _, time_consumption = result_summary[1]
print(f"Condition(get dummy method): use modified function")
print(f"Condition(number of columns): only 1 column is used to predict Salary")
print(f"The column to predict the best r2score: {rank_test[0][1]}, r2score: {round(rank_test[0][0], 3)}")
print(f"Time consumption: {round(time_consumption, 1)} seconds")

Condition(get dummy method): use modified function
Condition(number of columns): only 1 column is used to predict Salary
The column to predict the best r2score: Currency, r2score: 0.417
Time consumption: 1.0 seconds


In [133]:
condition, rank_test, _, time_consumption = result_summary[2]
print(f"Condition(get dummy method): use original get_dummies function")
print(f"Condition(number of columns): 2~{X.shape[1]} columns are used to predict Salary")
print(f"The number of columns to predict the best r2score: {rank_test[0][1]}, r2score: {round(rank_test[0][0], 3)}")
print(f"Time consumption: {round(time_consumption, 1)} seconds")

Condition(get dummy method): use original get_dummies function
Condition(number of columns): 2~145 columns are used to predict Salary
The number of columns to predict the best r2score: 4, r2score: 0.591
Time consumption: 241.9 seconds


In [132]:
condition, rank_test, _, time_consumption = result_summary[3]
print(f"Condition(get dummy method): use original modified function")
print(f"Condition(number of columns): 2~{X.shape[1]} columns are used to predict Salary")
print(f"The number of columns to predict the best r2score: {rank_test[0][1]}, r2score: {round(rank_test[0][0], 3)}")
print(f"Time consumption: {round(time_consumption, 1)} seconds")

Condition(get dummy method): use original modified function
Condition(number of columns): 2~145 columns are used to predict Salary
The number of columns to predict the best r2score: 5, r2score: 0.606
Time consumption: 3.9 seconds


# Q3. What can be done to improve the results?
#### Add one column at a time in the order of rank and check the result

In [135]:
import numpy as np

rank_test_simple = np.array(result_summary[0][1])[:, 1]
rank_test_my = np.array(result_summary[1][1])[:, 1]

In [142]:
def get_cols_gives_best_results(dict_num, dict_dummy, ranks):
    best_cols = []
    features = []
    best_score = 0

    for rank in ranks:
        features.append(rank)

        X = pd.DataFrame()
        for feature in features:
            if feature in dict_num:
                X = pd.concat([X, dict_num[feature]], axis=1)
            elif feature in dict_dummy:
                X = pd.concat([X, dict_dummy[feature]], axis=1)
            else:
                print(f"{feature} not in numeric_df_dict or categorized_df_dict!!!")
        test_score, train_score = check_result(len(features), X, y)

        if test_score > best_score:
            best_cols.append(rank)
            best_score = test_score
        else:
            features.pop(-1)

    print(f"Best score: {round(best_score, 3)}, Number of features: {len(features)}")
#     print(f"Best features: {best_cols}")

### Simple dummy results

In [146]:
start_time = time.time()
get_cols_gives_best_results(numeric_df_dict, dummy_simple_df_dict, rank_test_simple)
print(f"Time Consumption: {time.time()-start_time}")

Best score: 0.659, Number of features: 27
Time Consumption: 88.75718545913696


### My dummy results

In [147]:
start_time = time.time()
get_cols_gives_best_results(numeric_df_dict, dummy_my_df_dict, rank_test_my)
print(f"Time Consumption: {time.time()-start_time}")

Best score: 0.62, Number of features: 8
Time Consumption: 14.311509132385254


# Q4. Comparison conclusion

### There was no significant difference in the r2 score of Method 1 and Method 2.
### A big difference was found in the time consumption
### The time consumed for method 2 was found to be less than 1/60 to 1/6 compared to method 1


In [149]:
# single column regression time consumption
time1 = result_summary[0][3]
time2 = result_summary[1][3]
print(f"Method 1: {time1}, Method 2: {time2}, Method 2/Method 1: {time2/time1}")

Method 1: 53.780603885650635, Method 2: 0.9999985694885254, Method 2/Method 1: 0.018594037575605172


In [150]:
# multiple columns regression time consumption
time3 = result_summary[2][3]
time4 = result_summary[3][3]
print(f"Method 1: {time3}, Method 2: {time4}, Method 2/Method 1: {time4/time3}")

Method 1: 241.86028480529785, Method 2: 3.8835387229919434, Method 2/Method 1: 0.01605695092155484


In [ ]:
# time consumption when finding best result 
time3 = result_summary[2][3]
time4 = result_summary[3][3]
print(f"Method 1: {time3}, Method 2: {time4}, Method 2/Method 1: {time4/time3}")